In [ ]:
!pip install langchain langchain_community openai arxiv chromadb faiss-cpu python-dotenv

In [ ]:
from langchain.tools import Tool
import arxiv
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY']=userdata.get('OPENAI_API_KEY')

In [ ]:
def search_arxiv(query: str):
  results=arxiv.Search(query=query,max_results=3).results()
  return "\n\n".join([f"{r.title}\n{r.summary}" for r in results])

In [ ]:
arxiv_tool=Tool(
    name='ArxivSearch',
    func=search_arxiv,
    description='Search and summarize research papers from arxiv'
)

In [ ]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory

In [ ]:
llm=ChatOpenAI(temperature=0,model='gpt-4')
memory=ConversationBufferMemory(memory_key="chat_history",return_messages=True)
agent=initialize_agent(
    tools=[arxiv_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory
)

In [ ]:
agent.run("Find 3 recent papers on quantum computing and explain them to me like I'm 15.")



> Entering new AgentExecutor chain...
I need to search for recent papers on quantum computing and then summarize them in a simple and understandable way.
Action: ArxivSearch
Action Input: "recent quantum computing"
Observation: Quantum computer: an appliance for playing market games
Recent development in quantum computation and quantum information theory
allows to extend the scope of game theory for the quantum world. The authors
have recently proposed a quantum description of financial market in terms of
quantum game theory. The paper contain an analysis of such markets that shows
that there would be advantage in using quantum computers and quantum
strategies.

From Distributed Quantum Computing to Quantum Internet Computing: an Overview
The possibility of quantum computing has been proposed decades ago, at least
as far back as the 1980s, and distributed quantum computing has been studied
around two decades ago. Recent times have seen experimental successes and
advances in quantum c

/tmp/ipython-input-3-1550821245.py:2: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  results=arxiv.Search(query=query,max_results=3).results()


I have found three recent papers on quantum computing. Now I need to summarize them in a simple way.

Final Answer: 

1. "Quantum computer: an appliance for playing market games" - This paper talks about how quantum computers can be used in financial markets. Just like how we play video games, these computers can play "market games". They can make decisions and strategies that could be better than what we can do now.

2. "From Distributed Quantum Computing to Quantum Internet Computing: an Overview" - This paper is about how quantum computers are not just single machines, but can be connected together, like computers on the internet. This is called distributed quantum computing. The paper also talks about the progress in this field and how it's leading to something called the "quantum Internet".

3. "A Note on Bulk Quantum Turing Machine" - This paper discusses a specific type of quantum computer called a "bulk quantum Turing machine". It's like a normal computer, but it uses quantum m

'1. "Quantum computer: an appliance for playing market games" - This paper talks about how quantum computers can be used in financial markets. Just like how we play video games, these computers can play "market games". They can make decisions and strategies that could be better than what we can do now.\n\n2. "From Distributed Quantum Computing to Quantum Internet Computing: an Overview" - This paper is about how quantum computers are not just single machines, but can be connected together, like computers on the internet. This is called distributed quantum computing. The paper also talks about the progress in this field and how it\'s leading to something called the "quantum Internet".\n\n3. "A Note on Bulk Quantum Turing Machine" - This paper discusses a specific type of quantum computer called a "bulk quantum Turing machine". It\'s like a normal computer, but it uses quantum mechanics, the science of really small particles, to do its calculations. The paper talks about how this machine

In [ ]:
agent.run("What did I ask?")



> Entering new AgentExecutor chain...
The question asked for the process of answering a question using the tools provided, specifically ArxivSearch.
Final Answer: You asked for the process of answering a question using the tools provided, specifically ArxivSearch.

> Finished chain.


'You asked for the process of answering a question using the tools provided, specifically ArxivSearch.'

In [ ]:
print(memory.load_memory_variables({}))

{'chat_history': [HumanMessage(content="Find 3 recent papers on quantum computing and explain them to me like I'm 15.", additional_kwargs={}, response_metadata={}), AIMessage(content='1. "Quantum computer: an appliance for playing market games" - This paper talks about how quantum computers can be used in financial markets. Just like how we play video games, these computers can play "market games". They can make decisions and strategies that could be better than what we can do now.\n\n2. "From Distributed Quantum Computing to Quantum Internet Computing: an Overview" - This paper is about how quantum computers are not just single machines, but can be connected together, like computers on the internet. This is called distributed quantum computing. The paper also talks about the progress in this field and how it\'s leading to something called the "quantum Internet".\n\n3. "A Note on Bulk Quantum Turing Machine" - This paper discusses a specific type of quantum computer called a "bulk quan

## Adding a retriever tool to an Agent

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding_model=OpenAIEmbeddings()

vectorstore = Chroma(
    collection_name="research_memory",
    embedding_function=embedding_model,
    persist_directory="./chroma_db"
)

In [ ]:
def store_arxiv_results(query: str):
    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=3)

    texts = []
    metadatas = []

    for result in client.results(search):
        title = result.title
        summary = result.summary
        url = result.entry_id

        full_text = f"{title}\n\n{summary}"
        texts.append(full_text)
        metadatas.append({"source": "arxiv", "title": title, "url": url})

    if texts:
        vectorstore.add_texts(texts=texts, metadatas=metadatas)
        vectorstore.persist()
        print(f"Stored {len(texts)} papers in vector memory.")
    else:
        print("No results found.")


In [ ]:
store_arxiv_results("quantum computing superconducting qubits")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

Stored 3 papers in vector memory.


In [ ]:
retriever_tool=Tool(
    name='ResearchMemorySearch',
    func=lambda q: "\n\n".join([doc.page_content for doc in retriever.get_relevant_documents(q)]),
    description='Useful for answering academic or technical questions by searching stored research papers in memory.'
)

In [ ]:
memory1=ConversationBufferMemory(memory_key="chat_history",return_messages=True)
agent=initialize_agent(
    tools=[retriever_tool],
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,#AgentType.ZERO_SHOT_REACT_DESCRIPTION,#AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory1
)

In [ ]:
agent.invoke({"input": "Give me a quick overview of stored research related to quantum entanglement."})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: ResearchMemorySearch
Action Input: quantum entanglement
Observation: Universal quantum computing with superconducting charge qubits

Superconducting quantum circuit is a promising system for building quantum
computer. With this system we demonstrate the universal quantum computations,
including the preparing of initial states, the single-qubit operations, the
two-qubit universal gate operations between arbitrary qubits, the multiple
pairs of two-qubit gate operations in parallel, the coupling operations on a
group of qubits in parallel, the coupling operations on multiple groups of
qubits in parallel, the coupling operations on multiple pairs and multiple
groups of qubits in parallel. Within available technology, a universal quantum
computer consists of more than 50 qubits allowing operations is achievable with
the design.

Universal quantum computing with superconducting charge qubits

Superconductin

{'input': 'Give me a quick overview of stored research related to quantum entanglement.',
 'chat_history': [HumanMessage(content='Give me a quick overview of stored research related to quantum entanglement.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Here are some key findings from the stored research related to quantum entanglement:\n\n1. Superconducting quantum circuit is a promising system for building a quantum computer. It allows for universal quantum computations, including the preparation of initial states, single-qubit operations, two-qubit universal gate operations between arbitrary qubits, and various parallel operations. With current technology, a universal quantum computer with more than 50 qubits is achievable.\n\n2. The "Majorana superconducting qubit" is a proposed platform for universal quantum computation. It uses conventional s-wave superconducting leads to address a topological qubit stored in spatially separated Majorana bound states in a mul

In [ ]:
agent.invoke({"input": "What did I ask for?"})



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: You asked for a quick overview of stored research related to quantum entanglement.

> Finished chain.


{'input': 'What did I ask for?',
 'chat_history': [HumanMessage(content='Give me a quick overview of stored research related to quantum entanglement.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Here are some key findings from the stored research related to quantum entanglement:\n\n1. Superconducting quantum circuit is a promising system for building a quantum computer. It allows for universal quantum computations, including the preparation of initial states, single-qubit operations, two-qubit universal gate operations between arbitrary qubits, and various parallel operations. With current technology, a universal quantum computer with more than 50 qubits is achievable.\n\n2. The "Majorana superconducting qubit" is a proposed platform for universal quantum computation. It uses conventional s-wave superconducting leads to address a topological qubit stored in spatially separated Majorana bound states in a multi-terminal topological superconductor island. The abilit

In [ ]:
print(memory1.load_memory_variables({}))

{'chat_history': [HumanMessage(content='Give me a quick overview of stored research related to quantum entanglement.', additional_kwargs={}, response_metadata={}), AIMessage(content='Here are some key findings from the stored research related to quantum entanglement:\n\n1. Superconducting quantum circuit is a promising system for building a quantum computer. It allows for universal quantum computations, including the preparation of initial states, single-qubit operations, two-qubit universal gate operations between arbitrary qubits, and various parallel operations. With current technology, a universal quantum computer with more than 50 qubits is achievable.\n\n2. The "Majorana superconducting qubit" is a proposed platform for universal quantum computation. It uses conventional s-wave superconducting leads to address a topological qubit stored in spatially separated Majorana bound states in a multi-terminal topological superconductor island. The ability to turn on and off tunnel couplin